<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c3f5eb9c3ce19cb41c89236ad59592d8052670c301dd7891b2587456d03d890d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-05 15:45:17
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: -22.07 K (-0.15%)
Current PnL: -24.94 L (-15.91%)
CY Booked + Current PnL: -10.65 L (-6.79%)
-------------------
Total profit:  1.91 L
Total loss:  -26.85 L
-------------------
Total Booked + Current PnL: 16.33 L (12.58%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.91%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.91 L (62.38%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,3.26,15.53,3.60,19.69,7975.0,29778.0,221520.0,4.31,69.0,X-MC,3.04,78.0,3.73,1.54,23.38,XY25,NTT,AC
51,NESTLEIND,1377.00,2.73,15.90,4.75,21.40,14612.0,42190.0,307616.0,7.47,70.0,X-LC,5.02,12.0,2.89,2.13,23.23,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,1.91,-18.63,23.01,0.09,18868.0,-18778.0,81999.0,90.39,48.0,M-SC,3.48,253.0,-1.00,0.57,7.73,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.11,5.11,13.42,19.23,21225.0,7696.0,158156.0,22.81,83.0,L-LC,5.02,179.0,0.36,1.10,27.48,XY25,ATH,MINING
49,MASFIN,398.61,-1.46,-1.85,24.35,22.05,23416.0,-1815.0,96165.0,-15.54,53.0,H-SC,6.16,164.0,-0.08,0.67,39.78,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,8.04,-32.10,105.60,39.61,73498.0,-32900.0,69600.0,-54.11,56.0,H-SC,32.78,135.0,-0.45,0.48,12.66,AR,ATH,MISC
82,VOLTAS,1530.00,3.26,15.53,3.60,19.69,7975.0,29778.0,221520.0,4.31,69.0,X-MC,3.04,78.0,3.73,1.54,23.38,XY25,NTT,AC
51,NESTLEIND,1377.00,2.73,15.90,4.75,21.40,14612.0,42190.0,307616.0,7.47,70.0,X-LC,5.02,12.0,2.89,2.13,23.23,XY25,NTT,FMCG
0,5PAISA,593.00,2.12,-28.49,57.88,12.90,90438.0,-62246.0,156250.0,130.77,70.0,H-SC,11.38,161.0,-0.69,1.08,29.92,OX40N,NTT,FINANCE
34,HONAUT,58357.33,2.10,-15.69,69.99,43.32,96110.0,-25552.0,137320.0,-27.57,55.0,X-SC,7.75,90.0,-0.27,0.95,5.44,X40N,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJESHEXPO,518.0,-4.09,-66.47,198.40,0.07,91266.0,-91176.0,46001.0,1585.61,36.0,L-SC,4.26,268.0,-1.00,0.32,14.41,OX40N,NTT,JEWELLERY
18,COFFEEDAY,80.0,-2.94,-40.33,126.69,35.27,85841.0,-45792.0,67757.0,-53.72,48.0,L-SC,4.72,270.0,-0.53,0.47,65.84,XR,NTT,HOTELS
42,IRCTC,1269.0,-2.78,-18.59,87.85,52.93,193471.0,-50279.0,220229.0,-39.76,45.0,H-MC,6.48,118.0,-0.26,1.53,2.15,XY24,BTT,TRAVEL
80,VALIANTORG,838.0,-2.27,-43.53,208.37,74.15,192523.0,-71210.0,92395.0,-265.94,50.0,H-SC,21.08,149.0,-0.37,0.64,18.44,XR,NTT,CHEMICALS
84,WIPRO,420.0,-2.12,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,53.0,M-LC,5.66,99.0,0.12,1.27,14.66,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.32,-0.96,109.93,107.91,160745.0,-1419.0,146225.0,-19.34,56.0,M-SC,10.66,216.0,-0.01,1.01,9.03,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.32,-0.96,109.93,107.91,160745.0,-1419.0,146225.0,-19.34,56.0,M-SC,10.66,216.0,-0.01,1.01,9.03,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.16,-15.84,18.84,0.02,27679.0,-27644.0,146915.0,123.77,50.0,M-SC,7.00,234.0,-1.00,1.02,27.79,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.91,-18.63,23.01,0.09,18868.0,-18778.0,81999.0,90.39,48.0,M-SC,3.48,253.0,-1.00,0.57,7.73,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.73,-20.13,42.07,13.47,90614.0,-54279.0,215388.0,-67.15,58.0,H-SC,5.67,158.0,-0.60,1.49,9.52,XY24,NTT,PAINTS
66,SIS,528.00,-0.30,-22.31,56.07,21.26,48369.0,-24766.0,86266.0,2037.82,57.0,H-SC,4.46,166.0,-0.51,0.60,16.55,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.09,3.14,17.06,20.74,35749.0,6380.0,209550.0,35.83,73.0,M-MC,7.38,189.0,0.18,1.45,14.09,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.11,5.11,13.42,19.23,21225.0,7696.0,158156.0,22.81,83.0,L-LC,5.02,179.0,0.36,1.10,27.48,XY25,ATH,MINING
84,WIPRO,420.00,-2.12,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,53.0,M-LC,5.66,99.0,0.12,1.27,14.66,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-2.12,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,53.0,M-LC,5.66,99.0,0.12,1.27,14.66,XR,NTT,IT
25,FINCABLES,1641.55,-0.32,-0.96,109.93,107.91,160745.0,-1419.0,146225.0,-19.34,56.0,M-SC,10.66,216.0,-0.01,1.01,9.03,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.98,-5.50,118.75,106.72,138406.0,-6784.0,116552.0,-53.30,41.0,H-SC,6.65,139.0,-0.05,0.81,16.59,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.55,-2.78,39.51,35.63,80502.0,-5823.0,203751.0,-16.30,49.0,H-MC,2.91,119.0,-0.07,1.41,13.43,AR,ATH,PHARMA
49,MASFIN,398.61,-1.46,-1.85,24.35,22.05,23416.0,-1815.0,96165.0,-15.54,53.0,H-SC,6.16,164.0,-0.08,0.67,39.78,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.10,-14.29,29.25,10.78,60247.0,-34345.0,205973.0,-48.92,15.0,X-LC,3.78,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
23,DMART,5391.45,-1.98,-12.35,47.86,29.60,104707.0,-30822.0,218778.0,-25.28,27.0,X-LC,9.40,19.0,-0.29,1.52,7.12,X40N,ATH,FMCG
22,DIXON,18931.72,-0.94,-16.45,57.10,31.26,110098.0,-37952.0,192816.0,-60.98,31.0,X-MC,17.52,56.0,-0.34,1.34,2.41,X40N,ATH,IT
7,AWL,485.00,-1.05,-24.22,107.44,57.19,265513.0,-79000.0,247127.0,-64.14,32.0,X-MC,7.74,58.0,-0.30,1.71,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.63,-10.02,45.16,30.62,96326.0,-23754.0,213300.0,-35.98,34.0,X-MC,15.67,55.0,-0.25,1.48,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.44,1.04,19.52,20.77,46182.0,2442.0,236586.0,-7.76,39.0,X-MC,0.99,62.0,0.05,1.64,8.03,X40N,NTT,BREWERIES
65,SIEMENS,4671.50,-0.36,-17.07,51.35,25.51,79248.0,-31765.0,154330.0,-5.19,44.0,H-LC,1.31,49.0,-0.40,1.07,13.73,AR,ATH,ELECTRICAL
20,DABUR,735.00,-0.27,1.90,41.02,43.70,103691.0,4714.0,252782.0,-6.50,67.0,X-MC,1.87,72.0,0.05,1.75,17.24,XY24,BTT,FMCG
12,BERGEPAINT,680.00,0.86,-5.83,28.64,21.14,61313.0,-13256.0,214083.0,-12.56,41.0,X-MC,2.21,74.0,-0.22,1.49,18.89,XY24,NTT,PAINTS
81,VBL,671.64,-1.03,-1.28,37.43,35.67,116703.0,-4051.0,311791.0,-12.96,59.0,X-LC,2.55,4.0,-0.03,2.16,12.39,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.05,-24.22,107.44,57.19,265513.0,-79000.0,247127.0,-64.14,32.0,X-MC,7.74,58.0,-0.30,1.71,0.00,XY24,NTT,FMCG
43,ITC,452.00,-0.10,-14.29,29.25,10.78,60247.0,-34345.0,205973.0,-48.92,15.0,X-LC,3.78,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
52,PAGEIND,51605.08,-0.63,-10.02,45.16,30.62,96326.0,-23754.0,213300.0,-35.98,34.0,X-MC,15.67,55.0,-0.25,1.48,0.00,X40,ATH,APPARELS
19,COLPAL,3726.84,0.06,-20.59,78.21,41.51,163561.0,-54235.0,209130.0,-5.42,43.0,X-MC,8.59,61.0,-0.33,1.45,1.85,XY25,ATH,FMCG
11,BATAINDIA,2096.00,0.74,-38.26,118.98,35.20,94521.0,-49227.0,79443.0,4.12,45.0,X-SC,17.15,92.0,-0.52,0.55,2.04,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.93,-28.23,99.03,42.84,46201.0,-18352.0,46654.0,-53.17,51.0,X-SC,29.85,83.0,-0.40,0.32,5.55,XY24,NTT,MISC
58,RELAXO,1176.00,0.73,-47.39,189.48,52.31,144715.0,-68785.0,76375.0,-43.96,49.0,X-SC,7.46,91.0,-0.48,0.53,2.23,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.74,-38.26,118.98,35.20,94521.0,-49227.0,79443.0,4.12,45.0,X-SC,17.15,92.0,-0.52,0.55,2.04,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,1.68,3.33,20.08,24.08,26340.0,4225.0,131175.0,-5.01,62.0,X-SC,7.18,89.0,0.16,0.91,23.60,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,2.10,-15.69,69.99,43.32,96110.0,-25552.0,137320.0,-27.57,55.0,X-SC,7.75,90.0,-0.27,0.95,5.44,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-1.06,-11.57,36.50,20.71,122083.0,-43763.0,334474.0,-23.95,49.0,X-LC,2.71,1.0,-0.36,2.32,11.76,X40,ATH,IT
41,INFY,1972.00,-2.07,9.36,22.76,34.24,77876.0,29275.0,342163.0,-14.93,49.0,X-LC,7.41,2.0,0.38,2.37,18.41,X40,NTT,IT
75,TMPV,600.00,0.86,-23.44,60.62,22.98,104845.0,-52944.0,172954.0,-20.63,64.0,X-LC,5.23,3.0,-0.50,1.20,8.78,XY24,NTT,AUTO
81,VBL,671.64,-1.03,-1.28,37.43,35.67,116703.0,-4051.0,311791.0,-12.96,59.0,X-LC,2.55,4.0,-0.03,2.16,12.39,X40N,ATH,FMCG
43,ITC,452.00,-0.10,-14.29,29.25,10.78,60247.0,-34345.0,205973.0,-48.92,15.0,X-LC,3.78,5.0,-0.57,1.43,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.28,-33.56,100.01,32.89,51302.0,-25910.0,51297.0,-733.49,70.0,L-MC,6.89,259.0,-0.51,0.36,41.27,XR,NTT,BANKS
49,MASFIN,398.61,-1.46,-1.85,24.35,22.05,23416.0,-1815.0,96165.0,-15.54,53.0,H-SC,6.16,164.0,-0.08,0.67,39.78,XR,ATH,FINANCE
13,BSOFT,831.70,-1.12,-22.55,94.69,50.79,103151.0,-31717.0,108936.0,-0.66,51.0,H-SC,10.80,151.0,-0.31,0.76,26.84,XR,ATH,IT
69,SURYODAY,216.00,-0.88,-18.72,51.37,23.03,74771.0,-33517.0,145554.0,57.89,56.0,H-SC,5.54,167.0,-0.45,1.01,44.42,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,1.16,-15.84,18.84,0.02,27679.0,-27644.0,146915.0,123.77,50.0,M-SC,7.00,234.0,-1.00,1.02,27.79,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,2.12,-28.49,57.88,12.90,90438.0,-62246.0,156250.0,130.77,70.0,H-SC,11.38,161.0,-0.69,1.08,29.92,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-0.11,5.11,13.42,19.23,21225.0,7696.0,158156.0,22.81,83.0,L-LC,5.02,179.0,0.36,1.10,27.48,XY25,ATH,MINING
74,TITAGARH,1548.00,-0.79,-16.84,75.24,45.74,151536.0,-40773.0,201404.0,10.76,60.0,H-SC,3.37,174.0,-0.27,1.40,30.99,XY24,NTT,ENGINEERING
82,VOLTAS,1530.00,3.26,15.53,3.60,19.69,7975.0,29778.0,221520.0,4.31,69.0,X-MC,3.04,78.0,3.73,1.54,23.38,XY25,NTT,AC
36,ICICIPRULI,790.00,0.82,13.16,15.52,30.73,34600.0,25930.0,222935.0,-13.35,75.0,X-MC,5.41,75.0,0.75,1.55,27.61,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,43.97
2,50,76.34


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.89
MC    30.85
LC    26.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      23.75
X40N     12.75
XY25     10.73
XR        8.93
AR        8.59
OX40N     7.54
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.38
H-SC    22.81
X-LC    22.78
M-SC    11.37
X-SC     7.92
H-MC     4.66
M-MC     1.45
M-LC     1.27
L-LC     1.10
H-LC     1.07
L-SC     0.79
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.95,-7.71,43.04
IT,12.63,-17.38,76.58
FINANCE,11.12,-11.17,57.83
MISC,7.03,-28.12,80.53
ELECTRICAL,5.85,-11.14,51.84
PAINTS,5.72,-12.03,28.68
INSURANCE,4.74,0.61,33.40
PHARMA,3.94,-3.26,35.76
AUTO,2.86,-26.92,70.92


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3119504.0,21
AR,1330589.0,10
XR,1206652.0,12
X40,1097037.0,15
X40N,904533.0,9
OX40N,705802.0,10
XY25,345134.0,7
SR,281559.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3506425.0,24
X-MC,1470878.0,16
M-SC,1418573.0,15
X-LC,937731.0,13
X-SC,776947.0,8
H-MC,406725.0,3
L-SC,177107.0,2
M-LC,108900.0,1
H-LC,79248.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1219604.0      6
           AR         911569.0      5
M-SC       XY24       810603.0      6
H-SC       XR         784003.0      7
X-MC       X40        491847.0      7
           XY24       430517.0      3
X-LC       X40        420371.0      6
X-MC       X40N       376978.0      4
H-SC       OX40N      309690.0      4
X-SC       X40N       306145.0      3
M-SC       OX40N      304846.0      5
X-SC       XY24       285983.0      3
H-SC       SR         281559.0      2
X-LC       X40N       221410.0      2
H-MC       AR         213254.0      2
           XY24       193471.0      1
X-SC       X40        184819.0      2
X-LC       XY24       179326.0      2
M-SC       XR         176606.0      2
X-MC       XY25       171536.0      2
M-SC       AR         126518.0      2
X-LC       XY25       116624.0      3
M-LC       XR         108900.0      1
L-SC       OX40N       91266.0      1
           XR          85841.0      1
H-LC       AR          79248.0      1
L-MC       XR          51302.0      1
M-MC       XY25        35749.0      1
L-LC       XY25        21225.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
